In [ ]:
from database import Database
import uuid

In [ ]:
db = Database()
db.connect()

In [ ]:
# Clears all previous database entries
db.reset_tables()

In [ ]:
db.insert_user("first_test_user", "password123")
db.insert_user("second_test_user", "password456")

In [ ]:
prompt_id_1 = str(uuid.uuid4())
prompt_id_2 = str(uuid.uuid4())
image_id_1 = str(uuid.uuid4())
image_id_2 = str(uuid.uuid4())
image_id_3 = str(uuid.uuid4())

db.insert_chat("First Test Chat", 1)
db.insert_chat("Second Test Chat", 2)

db.insert_prompt(prompt_id_1, 1, 1, "a test prompt", 1, False, False, None, False, None, None, None)
db.insert_prompt(prompt_id_1, 1, 1, "another test prompt", 2, True, False, "suggestion text", True, "enhanced version", None, None)
db.insert_prompt(prompt_id_2, 2, 2, "yet another test prompt", 1, False, False, None, False, None, None, None)

db.insert_image(image_id_1, 1, 1, 7.0, 0.375, "images/test_image.jpg", input_prompt_id=1, output_prompt_id=None)
db.insert_image(image_id_2, 1, 1, 3.5, 0.5, "images/test_image2.jpg", input_prompt_id=1, output_prompt_id=2)
db.insert_image(image_id_2, 1, 2, 5.75, 0.3, "images/test_image3.jpg", input_prompt_id=None, output_prompt_id=2)
db.insert_image(image_id_3, 2, 9.0, 0.15, "images/test_image4.jpg", input_prompt_id=2, output_prompt_id=1)

In [ ]:
db.fetch_all_prompts()

In [ ]:
# Default for reading to pandas dataframe is True, change to False if you want to fetch without pandas

db.fetch_all_users(pandas=True)
# db.fetch_user_by_id(1)
# db.fetch_user_by_username("first_test_user")

In [ ]:
# Default for reading to pandas dataframe is True, change to False if you want to fetch without pandas

db.fetch_all_images(pandas=True)
# image = db.fetch_image_by_id(1)
# image = db.fetch_image_by_path("images/test_image.jpg")

In [ ]:
db.close()

# Authentication Examples

The database now supports users with passwords. Here are some authentication examples.

In [ ]:
# Authentication example
with Database() as db:
    # Register a new user
    db.insert_user("alice", "password123")

    # Verify login credentials
    user_data = db.fetch_user_by_username("alice")
    
    if not user_data.empty:
        stored_password = user_data.iloc[0]["password"]
        if "password123" == stored_password:
            print(f"Login successful for user: {user_data.iloc[0]['username']}")
        else:
            print("Incorrect password")
    else:
        print("User not found")